# Module

In [80]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np

In [81]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Dataset

In [82]:
data_path = '/home/xogns5037/딥러닝코딩스터디/RNN/Pytorch_RNN/IMDB_preproessing.csv'

In [83]:
df = pd.read_csv(data_path)
df.head()

,review,sentiment
0,one review ha mention watch 1 oz episod youll ...,1
1,wonder littl product film techniqu veri unassu...,1
2,thought thi wa wonder way spend time hot summe...,1
3,basic famili littl boy jake think zombi hi clo...,0
4,petter mattei love time money visual stun film...,1


## HoldOut

In [84]:
from sklearn.model_selection import train_test_split

In [85]:
x = df.review.values
y = df.sentiment.values

In [86]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1,stratify=y,shuffle=True)
x_train,x_valid,y_train,y_valid = train_test_split(x_train,y_train,test_size=0.2,stratify=y_train,shuffle=True)

In [87]:
print('Shape of x train : ',x_train.shape)
print('Shape of y train : ',y_train.shape)
print('Shape of x valid : ',x_valid.shape)
print('Shape of y valid : ',y_valid.shape)
print('Shape of x test : ',x_test.shape)
print('Shape of y test : ',y_test.shape)

Shape of x train :  (36000,)
Shape of y train :  (36000,)
Shape of x valid :  (9000,)
Shape of y valid :  (9000,)
Shape of x test :  (5000,)
Shape of y test :  (5000,)


## Tokenize

In [88]:
# Define tokenizer
tokenizer = Tokenizer(oov_token='<OOV>',num_words=10000)
# oov : out of vocabulary 즉, train 데이터에서는 등장하지 않았던 단어를 말한다.
# oov_token : 값이 지정된 경우, text_to_sequence 호출 과정에서 word_index에 추가되어 out-of-vocabulary words를 대체한다.
# oov_token='<OOV>' : 토큰화 과정에서 사전에 없는 단어가 발견되면 <OOV>로 대체한다.
# num_words : 단어 빈도가 많은 순서로 전체 단어의 개수를 10000개로 제한한다.

 - 모르는 단어의 경우에는 <OOV>라는 speical token으로 대체

 - 최빈도 단어 10000개 사용

In [96]:
# Train tokenizer
tokenizer.fit_on_texts(x_train) # fit_on_texts : 단어 인덱스를 구축한다.

우리는 valid set과 test set에 대해서는 정보를 알고 있으면 안됩니다. 그래서 train set에 대해서만 tokenizer를 학습해야 합니다. 그래야 train set에는 존재하지 않았지만 test set에 존재하는 경우 <OOV> token으로 변환하여 올바른 학습을 수행할 수 있습니다

In [97]:
# 단어 사전 출력

for word, idx in tokenizer.word_index.items():
  print(f'{word}\t\t=>\t{idx}')
  if idx > 10:
    break

<OOV>		=>	1
thi		=>	2
movi		=>	3
wa		=>	4
film		=>	5
hi		=>	6
one		=>	7
like		=>	8
ha		=>	9
time		=>	10
good		=>	11


In [98]:
# Length of token
len(tokenizer.word_index) # word_index : 단어와 정수 인덱스를 매핑한 딕셔너리

145293

In [99]:
# Tokenize
x_train_token = tokenizer.texts_to_sequences(x_train) # texts_to_sequences : 텍스트를 시퀀스로 변환한다.
x_valid_token = tokenizer.texts_to_sequences(x_valid)
x_test_token = tokenizer.texts_to_sequences(x_test)

In [101]:
print(x_train_token[0])

[364, 178, 165, 141, 836, 46, 43, 36, 4, 4393, 79, 3899, 468, 4393, 1553, 1329, 594, 4445, 828, 414, 1, 193, 100, 340, 1314, 5710, 659, 35, 8714, 90, 425, 2339, 269, 5710, 984, 3880, 9, 76, 9, 1, 7143, 5654, 148, 8, 17, 3, 8715, 2593, 1, 116, 3, 191, 381, 604, 4075, 3647, 160, 75, 3, 9, 4125, 72, 1863, 759, 105, 964, 173, 73, 273, 77, 296, 573, 1622, 54, 114, 519, 39, 31, 377, 111, 1, 854, 455, 1016, 1303, 3853, 164, 41, 355]


In [17]:
# Padding
x_train_pad = pad_sequences(x_train_token,maxlen=100,padding='post',truncating='post')
x_valid_pad = pad_sequences(x_valid_token,maxlen=100,padding='post',truncating='post')
x_test_pad = pad_sequences(x_test_token,maxlen=100,padding='post',truncating='post')

1차원 sequence에서 padding을 진행합니다.

우리가 가지고 있는 자연어는 거의 서로 다른 문장 길이를 가지고 있습니다. 이를 통일해주는 것이 필요한데 이에 대한 인자가 maxlen입니다. 저희는 문장의 최대 길이를 100으로 정하였습니다.

padding의 경우에는 문장의 길이가 96인 경우 최대 문장 길이인 100을 맞추기 위해 0을 4번 채우게 됩니다. 채우는 위치는 인자로 지정할 수 있습니다. post일 경우에는 뒤에서, pre일 경우에는 앞에서 채웁니다

문장의 최대 길이가 5라고 하면

1. padding = 'post'

[1,2,3,4]  -> [1,2,3,4,0]

2. padding = 'pre'

[1,2,3,4] -> [0,1,2,3,4]

truncating 같은 경우는 문장의 길이가 100을 넘는 경우 어떻게 자르는 지에 대한 인자입니다. post일 경우는 뒤에서 pre일 경우는 앞에서 자릅니다

문장의 최대 길이가 3이라고 한다면

1. truncating = 'post'

[1,2,3,4] -> [1,2,3]

2. truncating = 'pre'

[1,2,3,4] -> [2,3,4]

In [18]:
print(x_train_pad[0])

[  73   20    2    5   11   21 2124 7629    1  877 1318  263 4371   46
   46  443 6385 1644  185  891 8262 1048  339   93  278  573  401 3046
  990 1966  580    1   41 2979  108 1095 1033  375   28   40   18   36
  257   23  419 2142   12    2 6252  669   59 1588  299  124 5156 2445
 1414   16   28  162    2   47   60  654  948  193  338 8122    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [19]:
len(x_train_pad[0])

100

## Torch dataset

In [20]:
class TokenDataset(Dataset):

  def __init__(self, x, y):
    self.x = x
    self.y = y

  def __len__(self):
    return len(self.x)

  def __getitem__(self,idx):

    # 자연어처리에서는 해당 token들이 전부 정수형이므로 long type으로 변환해줍니다
    # 본격적인 학습은 embedding 층으로 변환된 값이 학습이 되는 겁니다
    x = torch.tensor(self.x[idx]).long()
    y = torch.tensor(self.y[idx]).long()
    return x,y

In [21]:
trainset = TokenDataset(x_train_pad,y_train)
validset = TokenDataset(x_valid_pad,y_valid)
testset = TokenDataset(x_test_pad,y_test)

In [22]:
# batch_size설정 기준은?
trainloader = DataLoader(trainset,batch_size=32,shuffle=True)
validloader = DataLoader(validset,batch_size=8)
testloader = DataLoader(testset,batch_size=4)

In [23]:
for x,y in trainloader:
  print(x)
  print(y)
  break

tensor([[ 328,    5,  152,  ...,    0,    0,    0],
        [ 183,   90, 1982,  ...,    0,    0,    0],
        [1492, 2096, 1783,  ...,    0,    0,    0],
        ...,
        [   2,    3, 1927,  ...,    0,    0,    0],
        [   2,  231,    8,  ...,    0,    0,    0],
        [ 193,  180, 1141,  ...,   13,    1,    1]])
tensor([0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 1, 1, 1, 0, 0])


# 모형

## RNN-base

In [27]:
class RNN(nn.Module):

  def __init__(self):
    super().__init__()
    # 단어들을 embedding 하는 것이기 때문에 단어의 수만큼을 우리가 원하는 차원으로 embedding 해주어야 합니다
    self.embedding = nn.Embedding(len(tokenizer.word_index),128)
    # 순환신경망 층입니다.
    self.rnn = nn.RNN(128,256,batch_first=True) # batch_first=True -> (배치 크기, 시퀀스 길이, 임베딩 차원)
    # 분류를 위한 선형층
    self.linear = nn.Linear(256,1)

  def forward(self,x):
    emb = self.embedding(x) # 입력 데이터 x를 임베딩합니다.
    h,o = self.rnn(emb) # 임베딩된 데이터를 RNN에 통과시킵니다. h : hidden state, o : output
    output = self.linear(h[:,-1,:]) # RNN의 마지막 시점의 hidden state를 선형층에 입력하여 분류 결과를 얻습니다.
    return output

forward 부분이 이해가 안 가실 수 있습니다 rnn은 두 가지 결과값을 뱉어냅니다. 하나는 RNN cell의 은닉 상태와 출력입니다. 우리는 출력만 필요하므로 출력을 가져다가 쓰게 됩니다. 아래에서 차근차근 보여드리겠습니다

그리고 우리는 손실함수를 BCEWithLogitsLoss()를 쓸 것인데 이는 이진 분류를 위한 손실함수입니다. 이진 분류를 하는 방법은 하나의 cell을 출력해서 0.5보다 크면 1 작으면 0으로 예측할 수 있습니다. 이를 위해서는 sigmoid 함수를 적용해주는 것이 맞지만 해당 손실함수를 작성하게 되면 자동적으로 sigmoid 함수를 적용시켜줍니다

In [28]:
for x,y in trainloader:
  sx = x
  sy = y
  break

In [29]:
sx

tensor([[ 243, 1461,   27,  ...,    0,    0,    0],
        [  96,   95,    2,  ...,    0,    0,    0],
        [   2,  252,  362,  ...,   20,  717,   10],
        ...,
        [  26,  316,    2,  ...,    0,    0,    0],
        [ 451,   51,  167,  ...,    0,    0,    0],
        [ 358,   10,  142,  ...,    0,    0,    0]])

In [30]:
emb = nn.Embedding(len(tokenizer.word_index),128)
emb(sx).shape

torch.Size([32, 100, 128])

In [125]:
emb(sx)[0]

tensor([[ 0.2149, -0.1013,  2.2706,  ..., -0.7170, -2.2248, -0.8596],
        [-0.2809,  1.2719, -0.2524,  ...,  2.0508, -1.0261,  0.8668],
        [-1.1893,  0.6626,  0.9921,  ...,  0.2013,  0.1428,  0.2079],
        ...,
        [-1.3802,  0.9942, -1.0217,  ..., -0.7228,  0.1280, -0.1162],
        [-1.3802,  0.9942, -1.0217,  ..., -0.7228,  0.1280, -0.1162],
        [-1.3802,  0.9942, -1.0217,  ..., -0.7228,  0.1280, -0.1162]],
       grad_fn=<SelectBackward0>)

100개의 토큰이 각각 하나의 128차원을 갖도록 embedding 되었습니다

In [31]:
o = nn.RNN(128,256,batch_first=True)(emb(sx))
o

(tensor([[[ 0.2612, -0.6099,  0.3281,  ..., -0.0778,  0.5042,  0.1674],
          [-0.0279,  0.0889, -0.3920,  ...,  0.4816,  0.7538, -0.0572],
          [-0.2276,  0.4893,  0.4059,  ...,  0.3801, -0.3176,  0.4900],
          ...,
          [-0.0700,  0.5305,  0.1893,  ..., -0.0625, -0.4179,  0.4662],
          [-0.0703,  0.5301,  0.1893,  ..., -0.0628, -0.4180,  0.4657],
          [-0.0703,  0.5302,  0.1893,  ..., -0.0626, -0.4179,  0.4657]],
 
         [[-0.3235, -0.5931,  0.6250,  ..., -0.5300,  0.7814, -0.4854],
          [-0.1787, -0.7040,  0.1378,  ...,  0.1955, -0.0039, -0.6996],
          [ 0.3632,  0.5239,  0.1589,  ...,  0.0554,  0.3725,  0.3217],
          ...,
          [-0.0701,  0.5302,  0.1893,  ..., -0.0625, -0.4179,  0.4658],
          [-0.0701,  0.5302,  0.1893,  ..., -0.0625, -0.4179,  0.4658],
          [-0.0701,  0.5302,  0.1893,  ..., -0.0625, -0.4179,  0.4658]],
 
         [[ 0.2220,  0.5452, -0.0283,  ..., -0.4150,  0.3233,  0.0638],
          [ 0.3247, -0.8905,

batch first는 우리가 학습할 때 보통 batch를 맨 앞에 두고 하는데 이를 위해 batch_first = True로 설정해주었습니다

그리고 출력값을 보게 되면 두 개의 tensor가 나왔습니다

In [32]:
o1,o2 = nn.RNN(128,256,batch_first=True)(emb(sx))
o1.shape, o2.shape

(torch.Size([32, 100, 256]), torch.Size([1, 32, 256]))

o1의 경우에는 각각의 rnn cell에서의 출력값들을 보관하고 있습니다. 100개의 token이 들어갔을 때 각각의 은닉 상태입니다.

o2는 RNN의 마지막 출력값이라고 생각하시면 됩니다.

결국에는 o1의 마지막 token 은닉 상태랑 o2랑 같은 값입니다

In [33]:
(o1[:,-1,:] == o2).all()

tensor(True)

In [34]:
nn.Linear(256,1)(o1[:,-1,:]).shape

torch.Size([32, 1])

In [35]:
preds = nn.Linear(256,1)(o1[:,-1,:]).squeeze()
preds

tensor([ 0.1812,  0.1812, -0.2161,  0.1812,  0.3454, -0.3201,  0.1812, -0.0999,
         0.1729, -0.0478, -0.4372,  0.3958,  0.1812,  0.1812, -0.3706,  0.1812,
         0.1845,  0.1812,  0.2164,  0.0354,  0.1812,  0.1812,  0.1812,  0.0337,
         0.0405,  0.1812,  0.1812,  0.1812,  0.1812,  0.1812,  0.1812,  0.1812],
       grad_fn=<SqueezeBackward0>)

In [36]:
final_output = nn.Sigmoid()(preds)
final_output

tensor([0.5452, 0.5452, 0.4462, 0.5452, 0.5855, 0.4206, 0.5452, 0.4750, 0.5431,
        0.4881, 0.3924, 0.5977, 0.5452, 0.5452, 0.4084, 0.5452, 0.5460, 0.5452,
        0.5539, 0.5089, 0.5452, 0.5452, 0.5452, 0.5084, 0.5101, 0.5452, 0.5452,
        0.5452, 0.5452, 0.5452, 0.5452, 0.5452], grad_fn=<SigmoidBackward0>)

In [37]:
final_output[final_output>=0.5] = 1
final_output[final_output<0.5] = 0

In [38]:
final_output

tensor([1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       grad_fn=<IndexPutBackward0>)

In [39]:
(final_output == sy).sum().item()

18

위의 부분이 이해가 안되시면 RNN의 구조를 한 번 다시 보시고, 그래도 어려우시다면 질문 주세요!

In [40]:
rnn = RNN().to(device)

In [41]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(rnn.parameters(),lr=0.0001)

In [42]:
def training(epoch, model, trainloader, validloader) :
    correct = 0
    total = 0
    running_loss = 0

    model.train()

    for x,y in trainloader :
        x = x.to(device)
        y = y.to(device).float() # 계산이 소수형끼리 연산이기에 바꿔주어야함
        y_pred = model(x).squeeze() #[32,1] -> [32]
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        with torch.no_grad() :
            y_pred[y_pred>=0.5] = 1
            y_pred[y_pred<0.5] = 0
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()

    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct / total

    valid_correct = 0
    valid_total = 0
    valid_running_loss = 0

    model.eval()
    with torch.no_grad() :
        for x,y in validloader :
            x = x.to(device)
            y = y.to(device).float()
            y_pred = model(x).squeeze()
            loss = loss_fn(y_pred, y)
            y_pred[y_pred>=0.5] = 1
            y_pred[y_pred<0.5] = 0
            valid_correct += (y_pred == y).sum().item()
            valid_total += y.size(0)
            valid_running_loss += loss.item()

    epoch_valid_loss = valid_running_loss / len(validloader.dataset)
    epoch_valid_acc = valid_correct / valid_total

    print('epoch :', epoch,
         'loss :', round(epoch_loss, 3),
         'accuarcy :', round(epoch_acc, 3),
         'valid_loss :', round(epoch_valid_loss,3),
         'valid_accuracy :', round(epoch_valid_acc, 3))

    return epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc

In [43]:
epochs = 5
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

for epoch in range(epochs) :
    epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc = training(epoch, rnn, trainloader, validloader)

    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    valid_loss.append(epoch_valid_loss)
    valid_acc.append(epoch_valid_acc)

epoch : 0 loss : 0.022 accuarcy : 0.5 valid_loss : 0.087 valid_accuracy : 0.5
epoch : 1 loss : 0.021 accuarcy : 0.515 valid_loss : 0.086 valid_accuracy : 0.504
epoch : 2 loss : 0.021 accuarcy : 0.528 valid_loss : 0.085 valid_accuracy : 0.54
epoch : 3 loss : 0.021 accuarcy : 0.531 valid_loss : 0.084 valid_accuracy : 0.547
epoch : 4 loss : 0.021 accuarcy : 0.549 valid_loss : 0.083 valid_accuracy : 0.566


In [44]:
# 평가
test_correct = 0
test_total = 0
test_running_loss = 0

rnn.eval()
with torch.no_grad() :
    for x,y in testloader :
        x = x.to(device)
        y = y.to(device).float()
        y_pred = rnn(x).squeeze()
        loss = loss_fn(y_pred, y)
        y_pred[y_pred>=0.5] = 1
        y_pred[y_pred<0.5] = 0
        test_correct += (y_pred == y).sum().item()
        test_total += y.size(0)
        test_running_loss += loss.item()

epoch_test_loss = test_running_loss / len(testloader.dataset)
epoch_test_acc = test_correct / test_total

In [45]:
print('Test Loss : ',epoch_test_loss)
print('Test Acc : ',epoch_test_acc)

Test Loss :  0.1656660062789917
Test Acc :  0.5668


## LSTM

In [46]:
class LSTM(nn.Module):

  def __init__(self):
    super().__init__()
    self.embedding = nn.Embedding(len(tokenizer.word_index),128)
    self.lstm = nn.LSTM(128,256,batch_first=True)
    self.linear = nn.Linear(256,1)

  def forward(self,x):
    emb = self.embedding(x)
    output,_ = self.lstm(emb)
    output = self.linear(output[:,-1,:])
    return output

In [47]:
o1,(o2,o3) = nn.LSTM(128,256,batch_first=True)(emb(sx))
o1.shape

torch.Size([32, 100, 256])

In [48]:
o2.shape, o3.shape #o2 : hidden state, o3 : cell state

(torch.Size([1, 32, 256]), torch.Size([1, 32, 256]))

LSTM은 vanilla RNN과 다르게 총 3 가지 출력값을 가집니다. LSTM은 vanilla RNN의 단점을 보완하기 위해서 cell state라는 장치를 추가하였습니다. 그러므로 LSTM의 출력값은 최종 출력값, 은닉상태, cell state 이렇게 3가지를 가지게 됩니다

감성 분류 시는 RNN과 동일하게 사용하면 됩니다

In [49]:
lstm = LSTM().to(device)
lstm.train()

LSTM(
  (embedding): Embedding(144816, 128)
  (lstm): LSTM(128, 256, batch_first=True)
  (linear): Linear(in_features=256, out_features=1, bias=True)
)

In [50]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(lstm.parameters(),lr=0.0001)

In [51]:
epochs = 10
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

for epoch in range(epochs) :
    epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc = training(epoch, lstm, trainloader, validloader)

    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    valid_loss.append(epoch_valid_loss)
    valid_acc.append(epoch_valid_acc)

epoch : 0 loss : 0.022 accuarcy : 0.501 valid_loss : 0.084 valid_accuracy : 0.525
epoch : 1 loss : 0.02 accuarcy : 0.581 valid_loss : 0.074 valid_accuracy : 0.689
epoch : 2 loss : 0.019 accuarcy : 0.675 valid_loss : 0.08 valid_accuracy : 0.694
epoch : 3 loss : 0.018 accuarcy : 0.698 valid_loss : 0.069 valid_accuracy : 0.732
epoch : 4 loss : 0.019 accuarcy : 0.651 valid_loss : 0.075 valid_accuracy : 0.725
epoch : 5 loss : 0.018 accuarcy : 0.67 valid_loss : 0.062 valid_accuracy : 0.761
epoch : 6 loss : 0.014 accuarcy : 0.78 valid_loss : 0.058 valid_accuracy : 0.768
epoch : 7 loss : 0.013 accuarcy : 0.804 valid_loss : 0.055 valid_accuracy : 0.772
epoch : 8 loss : 0.012 accuarcy : 0.824 valid_loss : 0.051 valid_accuracy : 0.815
epoch : 9 loss : 0.011 accuarcy : 0.842 valid_loss : 0.049 valid_accuracy : 0.808


In [52]:
# 평가
test_correct = 0
test_total = 0
test_running_loss = 0

lstm.eval()
with torch.no_grad() :
    for x,y in testloader :
        x = x.to(device)
        y = y.to(device).float()
        y_pred = lstm(x).squeeze()
        loss = loss_fn(y_pred, y)
        y_pred[y_pred>=0.5] = 1
        y_pred[y_pred<0.5] = 0
        test_correct += (y_pred == y).sum().item()
        test_total += y.size(0)
        test_running_loss += loss.item()

epoch_test_loss = test_running_loss / len(testloader.dataset)
epoch_test_acc = test_correct / test_total

In [53]:
print('Test Loss : ',epoch_test_loss)
print('Test Acc : ',epoch_test_acc)

Test Loss :  0.09851438860036432
Test Acc :  0.8008


## Stacked RNN

In [54]:
class StackedLSTM(nn.Module):

  def __init__(self):
    super().__init__()
    self.embedding = nn.Embedding(len(tokenizer.word_index),128)
    self.lstm = nn.LSTM(128,256,num_layers=2,batch_first=True) # LSTM 층을 두 번 쌓아줌
    self.linear = nn.Linear(256,1)

  def forward(self,x):
    emb = self.embedding(x)
    output,_ = self.lstm(emb)
    output = self.linear(output[:,-1,:])
    return output

In [55]:
stacked_lstm = StackedLSTM().to(device)

In [56]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(stacked_lstm.parameters(),lr=0.0001)

In [57]:
epochs = 10
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

for epoch in range(epochs) :
    epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc = training(epoch, stacked_lstm, trainloader, validloader)

    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    valid_loss.append(epoch_valid_loss)
    valid_acc.append(epoch_valid_acc)

epoch : 0 loss : 0.021 accuarcy : 0.519 valid_loss : 0.084 valid_accuracy : 0.5
epoch : 1 loss : 0.021 accuarcy : 0.5 valid_loss : 0.087 valid_accuracy : 0.5
epoch : 2 loss : 0.022 accuarcy : 0.5 valid_loss : 0.086 valid_accuracy : 0.5
epoch : 3 loss : 0.021 accuarcy : 0.51 valid_loss : 0.085 valid_accuracy : 0.5
epoch : 4 loss : 0.02 accuarcy : 0.591 valid_loss : 0.07 valid_accuracy : 0.724
epoch : 5 loss : 0.016 accuarcy : 0.734 valid_loss : 0.062 valid_accuracy : 0.755
epoch : 6 loss : 0.015 accuarcy : 0.767 valid_loss : 0.059 valid_accuracy : 0.767
epoch : 7 loss : 0.014 accuarcy : 0.792 valid_loss : 0.055 valid_accuracy : 0.79
epoch : 8 loss : 0.013 accuarcy : 0.811 valid_loss : 0.052 valid_accuracy : 0.802
epoch : 9 loss : 0.012 accuarcy : 0.827 valid_loss : 0.05 valid_accuracy : 0.808


In [58]:
# 평가
test_correct = 0
test_total = 0
test_running_loss = 0

stacked_lstm.eval()
with torch.no_grad() :
    for x,y in testloader :
        x = x.to(device)
        y = y.to(device).float()
        y_pred = stacked_lstm(x).squeeze()
        loss = loss_fn(y_pred, y)
        y_pred[y_pred>=0.5] = 1
        y_pred[y_pred<0.5] = 0
        test_correct += (y_pred == y).sum().item()
        test_total += y.size(0)
        test_running_loss += loss.item()

epoch_test_loss = test_running_loss / len(testloader.dataset)
epoch_test_acc = test_correct / test_total

In [59]:
print('Test Loss : ',epoch_test_loss)
print('Test Acc : ',epoch_test_acc)

Test Loss :  0.09815319024100899
Test Acc :  0.8186


## Bidirectional RNN

In [60]:
class BiLSTM(nn.Module):

  def __init__(self):
    super().__init__()
    self.embedding = nn.Embedding(len(tokenizer.word_index),128)
    self.lstm = nn.LSTM(128,256,num_layers=2,batch_first=True,bidirectional=True) # LSTM 층을 두 번 쌓아줌
    self.linear = nn.Linear(256*2,1) # 양방향일 경우 은닉 상태가 정방향과 역방향이 이어서 나오므로 2배를 해주어야 함

  def forward(self,x):
    emb = self.embedding(x)
    output,_ = self.lstm(emb)
    output = self.linear(output[:,-1,:])
    return output

In [61]:
bilstm = BiLSTM().to(device)

In [62]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(bilstm.parameters(),lr=0.0001)

In [63]:
epochs = 10
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

for epoch in range(epochs) :
    epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc = training(epoch, bilstm, trainloader, validloader)

    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    valid_loss.append(epoch_valid_loss)
    valid_acc.append(epoch_valid_acc)

epoch : 0 loss : 0.021 accuarcy : 0.537 valid_loss : 0.078 valid_accuracy : 0.643
epoch : 1 loss : 0.019 accuarcy : 0.631 valid_loss : 0.071 valid_accuracy : 0.689
epoch : 2 loss : 0.017 accuarcy : 0.719 valid_loss : 0.07 valid_accuracy : 0.679
epoch : 3 loss : 0.016 accuarcy : 0.754 valid_loss : 0.089 valid_accuracy : 0.666
epoch : 4 loss : 0.015 accuarcy : 0.769 valid_loss : 0.06 valid_accuracy : 0.762
epoch : 5 loss : 0.013 accuarcy : 0.806 valid_loss : 0.052 valid_accuracy : 0.791
epoch : 6 loss : 0.012 accuarcy : 0.831 valid_loss : 0.048 valid_accuracy : 0.819
epoch : 7 loss : 0.01 accuarcy : 0.849 valid_loss : 0.046 valid_accuracy : 0.829
epoch : 8 loss : 0.009 accuarcy : 0.868 valid_loss : 0.045 valid_accuracy : 0.841
epoch : 9 loss : 0.008 accuarcy : 0.882 valid_loss : 0.045 valid_accuracy : 0.839


In [64]:
# 평가
test_correct = 0
test_total = 0
test_running_loss = 0

bilstm.eval()
with torch.no_grad() :
    for x,y in testloader :
        x = x.to(device)
        y = y.to(device).float()
        y_pred = bilstm(x).squeeze()
        loss = loss_fn(y_pred, y)
        y_pred[y_pred>=0.5] = 1
        y_pred[y_pred<0.5] = 0
        test_correct += (y_pred == y).sum().item()
        test_total += y.size(0)
        test_running_loss += loss.item()

epoch_test_loss = test_running_loss / len(testloader.dataset)
epoch_test_acc = test_correct / test_total

In [65]:
print('Test Loss : ',epoch_test_loss)
print('Test Acc : ',epoch_test_acc)

Test Loss :  0.09286864044461399
Test Acc :  0.8376
